In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import pandas as pd
import numpy as np
from synthetic_neurons import SAMNeuron
import csv
from PIL import Image
import json

## Convert Neurons to required format
Input the mode of the synthetic neuron and the path to the saved images.

In [2]:
mode = 'mono'
data_path = "./dataset_exemplars_COCO_v3/"


data_path = os.path.join(data_path, mode)
num_exemplars = 15
data_dict = []
all_activations = []
all_images = []
labels = []
count_res = 0
base = os.path.join("./synthetic_neurons_v3", mode)

if os.path.exists(os.path.join(base, "data.json")):
    with open(os.path.join(base, "data.json"), 'r') as file:
        data_dict = json.load(file)
    all_activations = np.load(os.path.join(base, "activations.npy")).tolist()
    all_images = np.load(os.path.join(base, "images.npy")).tolist()
    all_masks = np.load(os.path.join(base, "masks.npy")).tolist()
    labels = np.load(os.path.join(base, "masks.npy")).tolist()
    count_res = len(all_images)

# Expects data in format:
# data_path/label
# For multi-label neurons:
# data_path/label1_label2
for x in sorted(os.listdir(data_path)[count_res:]):
    print("Processing Class: "+x)
    current_path = os.path.join(data_path, x)
    print(current_path)
    if not(os.path.isdir(current_path)) or ("ipynb_checkpoints" in x):
        print(x)
        continue
    labels = [x.rsplit('_')[1]]
    print(labels)
    neuron = SAMNeuron(labels=labels, mode=mode)
    os.makedirs(os.path.join(base, 'masks', x),exist_ok=True)
    os.makedirs(os.path.join(base, 'masked_images', x),exist_ok=True)
    os.makedirs(os.path.join(base, 'images', x),exist_ok=True)
    os.makedirs(base+"data/" +x,exist_ok=True)
    logits = []
    names = []
    images = []
    masks = []
    for y in os.listdir(current_path):
        print(y)
        if ("ipynb_checkpoints" in y):
            continue
        if y[-3:] == 'txt'or y[-3:] == 'csv': continue
        try:
            input_image = Image.open(os.path.join(current_path, y))
            [curr_logit], [image] = neuron.calc_activations([input_image])
            mask = Image.fromarray(255 * mask.astype(np.uint8))
            mask = mask.resize((224, 224), Image.Resampling.LANCZOS)
            mask.save(os.path.join(base, 'masks', x, y))
            masks.append(np.array(mask))
            print(curr_logit)
            inside = np.array(mask > 0)
            outside = np.array(mask == 0)
            masked_image = image * inside + 0 * image * outside
            display(masked_image)
            names.append(y)
            masked_image.save(os.path.join(base, 'masked_images', x, y))
            image = Image.fromarray(image.astype(np.uint8))
            image = image.resize((224, 224), Image.Resampling.LANCZOS)
            image.save(os.path.join(base, 'images', x, y))
            images.append(np.array(image))
            logits.append(curr_logit)
        except Exception as e:
            # embed()
            print(e)
            print("failed processing image "+"/"+x+"/"+y)                
            continue
        # count+=1
    indx = np.argsort(-np.array(logits))
    indx = indx[:num_exemplars]
    if len(indx)<num_exemplars: 
        print("not enough images "+"/" + x + "/" + y) 
        break
    activation_list = np.array(logits)[indx]
    image_list = np.array(names)[indx]
    data_dict.append({"label": x, "image_names": image_list.tolist(), "activations": activation_list.tolist()})
    all_activations.append(activation_list)
    all_images.append(np.array(images)[indx])
    all_masks.append(np.array(masks)[indx])
    labels.append(x)
    with open(os.path.join(base, "data.json"), 'w') as file:
        json.dump(data_dict, file, indent=4)
    np.save(os.path.join(base, "activations.npy"), np.array(all_activations))
    np.save(os.path.join(base, "images.npy"), np.array(all_images))
    np.save(os.path.join(base, "masks.npy"), np.array(all_masks))
    np.save(os.path.join(base, "labels.npy"), np.array(labels))

with open(os.path.join(base, "activations.csv"), 'w', newline='') as file:
    writer = csv.writer(file)
    for row in all_activations:
        writer.writerow(row)

Processing Class: 00_laptop
./dataset_exemplars_COCO_v3/Age/Young/00_laptop
['laptop']
final text_encoder_type: bert-base-uncased
12.png
1 faces detected
objected detected, biased face detected: 0.68
0.68
9.png
1 faces detected
objected detected, biased face detected: 0.71
0.71
0.png
3 faces detected
objected detected, biased face detected: 0.76
0.76
7.png
1 faces detected
objected detected, biased face detected: 0.73
0.73
14.png
1 faces detected
objected detected, biased face detected: 0.68
0.68
13.png
1 faces detected
objected detected, biased face detected: 0.68
0.68
6.png
2 faces detected
objected detected, biased face detected: 0.73
0.73
1.png
1 faces detected
objected detected, biased face detected: 0.75
0.75
8.png
1 faces detected
objected detected, biased face detected: 0.7066666666666667
0.71
2.png
2 faces detected
objected detected, biased face detected: 0.74
0.74
5.png
1 faces detected
objected detected, biased face detected: 0.74
0.74
10.png
5 faces detected
objected detect